# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True # also works
# nav = webdriver.Chrome(options=chrome_options)

nav = webdriver.Chrome()

# pesquisar cotação dolar
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# pesquisar cotação dolar
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pegar cotacao ouro
nav.get("https://www.melhorcambio.com/")
aba_original = nav.window_handles[0]
nav.find_element_by_xpath('//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img').click()
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova)
cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

nav.quit()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [13]:
import pandas as pd

produtos_df = pd.read_excel("Produtos.xlsx")
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [14]:
produtos_df.loc[produtos_df['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)

produtos_df['Preço Base Reais'] = produtos_df['Cotação'] * produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Ajuste'] * produtos_df['Preço Base Reais']
# produtos_df["Preço Final"] = produtos_df["Preço Final"].map("{:.2f}".format)
display(produtos_df)

NameError: name 'cotacao_dolar' is not defined

### Agora vamos exportar a nova base de preços atualizada

In [11]:
produtos_df.to_excel("Produtos Atualizado.xlsx", index=False)